### Assignment 4  
Minze Teh & Sebastian Kölbel

We begin by testing the perceptron class provided in the assignment, and we were able to run it

In [289]:
import numpy as np
from sklearn.base import BaseEstimator

class LinearClassifier(BaseEstimator):
    """
    General class for binary linear classifiers. Implements the predict
    function, which is the same for all binary linear classifiers. There are
    also two utility functions.
    """

    def decision_function(self, X):
        """
        Computes the decision function for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """
        return X.dot(self.w)

    def predict(self, X):
        """
        Predicts the outputs for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """

        # First compute the output scores
        scores = self.decision_function(X)

        # Select the positive or negative class label, depending on whether
        # the score was positive or negative.
        out = np.select([scores >= 0.0, scores < 0.0],
                        [self.positive_class,
                         self.negative_class])
        return out

    def find_classes(self, Y):
        """
        Finds the set of output classes in the output part Y of the training set.
        If there are exactly two classes, one of them is associated to positive
        classifier scores, the other one to negative scores. If the number of
        classes is not 2, an error is raised.
        """
        classes = sorted(set(Y))
        if len(classes) != 2:
            raise Exception("this does not seem to be a 2-class problem")
        self.positive_class = classes[1]
        self.negative_class = classes[0]

    def encode_outputs(self, Y):
        """
        A helper function that converts all outputs to +1 or -1.
        """
        return np.array([1 if y == self.positive_class else -1 for y in Y])

class Perceptron(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        # Perceptron algorithm:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):

                # Compute the output score for this instance.
                score = x.dot(self.w)

                # If there was an error, update the weights.
                if y*score <= 0:
                    self.w += y*x  

In [290]:
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# from aml_perceptron import Perceptron, SparsePerceptron

# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y

# Read all the documents.
X, Y = read_data('DAT341-Applied-Machine-Learning/PA4/data/all_sentiment_shuffled.txt')

# Split into training and test parts.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                random_state=0)
# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),
    
    Perceptron()
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 1.21 sec.
Accuracy: 0.7919.


We were able to run the code and recieved an accuracy of just under 0.8. We moved on to look at the Pegasus algorithm

In [291]:
class PegasusSVC(LinearClassifier):
    """
    An implementation of the Pegasus learning algorithm using the SVC approach.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter  = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
    
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        lambda_ = 1/n_samples

        # Pegasus algorithm:
        for t in range(1,self.n_iter+1):

            idx = np.random.randint(n_samples)

            eta = 1/(lambda_*t)

            x, y = X[idx], Ye[idx]

            # Compute the output score for this instance.
            score = self.w.dot(x)
            # If there was an error, update the weights.
            self.w *= 1-eta*lambda_
            
            if y*score < 1:
                self.w += x*eta*y

class PegasusLR(LinearClassifier):
    """
    An implementation of the pegasus learning algorithm using LR approach.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter  = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
    
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        lambda_ = 1/n_samples

        # Pegasus algorithm:
        for t in range(1,self.n_iter+1):

            idx = np.random.randint(n_samples)

            eta = 1/(lambda_*t)

            x, y = X[idx], Ye[idx]
            
            # calculate gradient of log Loss
            dLoss = -y/(1+np.exp(y*(self.w*x)))
            # calc gradient
            gradient = lambda_*self.w+dLoss
            # update step
            self.w -= eta*gradient

In [292]:
# Read all the documents.
X, Y = read_data('DAT341-Applied-Machine-Learning/PA4/data/all_sentiment_shuffled.txt')

# Split into training and test parts.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                random_state=0)
# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),
    
    PegasusSVC(n_iter=10000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 1.13 sec.
Accuracy: 0.8133.


In [293]:
# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),
    
    PegasusLR(n_iter=10000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 1.13 sec.
Accuracy: 0.4927.


Logistic regression has a lower accuracy compared to linearSVC as they utilize different optimization function. Logistic regression minimizes the log loss function  which models class probabilities directly, while linear SVC minimizes the hinge loss function,  maximising the margin between classes. This difference becomes significant when dealing with data that is not linearly separable, as logistic regression may struggle to find an appropriate decision boundary. Apart from this, logistic regression may be sensitive to class imbalances, potentially leading to biased predictions if one class is more apparent in the dataset.

We continued to optimize the algorithm using the BLAS functions as well as a approach using sparse vector operations.

In [294]:
import scipy.linalg.blas as blas

class PegasusSVC_BLAS(LinearClassifier):
    """
    An implementation of the pegasus learning algorithm using SVC and BLAS functions.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter  = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the pegasus learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
    
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        lambda_ = 1/n_samples

        # Pegasus algorithm:
        for t in range(1,self.n_iter+1):

            idx = np.random.randint(n_samples)

            eta = 1/(lambda_*t)

            x, y = X[idx], Ye[idx]

            # Compute the output score for this instance.
            score = blas.ddot(self.w,x)

            # If there was an error, update the weights.
            
            blas.dscal(1-eta*lambda_, self.w)
            if y*score < 1:
                
                blas.daxpy(x, self.w, a=eta*y)

def add_sparse_to_dense(x, w, factor):
    """
    Adds a sparse vector x, scaled by some factor, to a dense vector.
    This can be seen as the equivalent of w += factor * x when x is a dense
    vector.
    """
    w[x.indices] += factor * x.data

def sparse_dense_dot(x, w):
    """
    Computes the dot product between a sparse vector x and a dense vector w.
    """
    return np.dot(w[x.indices], x.data)

class SparsePegasusSVC(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm,
    assuming that the input feature matrix X is sparse.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # Initialize the weight vector to all zeros.
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
         
        # set lambda
        lambda_ = 1/n_samples

        # pegasus algorothm
        for t in range(1,self.n_iter+1):

            idx = np.random.randint(n_samples)

            eta = 1/(lambda_*t)

            x, y = X[idx], Ye[idx]

            # Compute the output score for this instance.
            score = sparse_dense_dot(x,self.w)

            # If there was an error, update the weights.
            self.w *= 1-eta*lambda_
            if y*score < 1:
                add_sparse_to_dense(x, self.w, eta*y)

In [295]:
# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(ngram_range=(1,2)),
    Normalizer(),
    PegasusSVC(n_iter=10000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
train_time = t1-t0
print('Training time: {:.2f} sec.'.format(train_time))

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 17.50 sec.
Accuracy: 0.8258.


In [296]:
# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(ngram_range=(1,2)),
    Normalizer(),
    PegasusSVC_BLAS(n_iter=10000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
train_time_blas = t1-t0
print('Training time: {:.2f} sec.'.format(train_time_blas))

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 11.06 sec.
Accuracy: 0.8405.


In [297]:
pipeline = make_pipeline(
    TfidfVectorizer(ngram_range=(1,2)),
    Normalizer(),
    SparsePegasusSVC(n_iter=10000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
train_time_sparse = t1-t0
print('Training time: {:.2f} sec.'.format(train_time_sparse))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 5.60 sec.
Accuracy: 0.8485.


In [298]:
class SparsePegasusSVC2(LinearClassifier):

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # Initialize the weight vector to all zeros.
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
         
        # set lambda
        self.lambda_ = 1/n_samples
        a = 1
        # pegasus algorithm
        for t in range(1,self.n_iter+1):

            idx = np.random.randint(n_samples)

            # For this approach the eta was modified slightly to prevent a from becoming 0.
            eta = 1.000001/(self.lambda_*t)

            x, y = X[idx], Ye[idx]

            # scaling step
            a *= 1-eta*self.lambda_
                  
            # Compute the output score for this instance.
            score = a * sparse_dense_dot(x,self.w)

            if y*score < 1: 
                add_sparse_to_dense(x, self.w, eta*y/a)
                
        # use accumulated factor to adjust the weights
        self.w *= a

In [299]:
pipeline = make_pipeline(
    TfidfVectorizer(ngram_range=(1,2)),
    Normalizer(),
    SparsePegasusSVC2(n_iter=10000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
train_time_sparse_2 = t1-t0
print('Training time: {:.2f} sec.'.format(train_time_sparse_2))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 3.54 sec.
Accuracy: 0.8410.


In [300]:
from IPython.display import Markdown as md

md("""Training times in sec
   |Pre-optimization|BLAS|Sparse|Sparse (scaling factor)|
   |--|--|--|--|
   |%.2f|%.2f|%.2f|%.2f| """ %(train_time,train_time_blas,train_time_sparse,train_time_sparse_2))

Training times in sec
   |Pre-optimization|BLAS|Sparse|Sparse (scaling factor)|
   |--|--|--|--|
   |17.50|11.06|5.60|3.54| 

It is visible in the table that the optimization steps performed yielded a shorter training time. with the sparse approach being the most time saving.